![image](competi.png)

#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

In [6]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn import svm
import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
from sklearn import metrics
import seaborn as sns
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder

In [7]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("hospital_train.csv",index_col=0)
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,
161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237869,12,a,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,More than 100 Days
254763,28,b,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,51-60
69788,6,a,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,31-40


In [8]:
df.drop(["1","4"],axis=1,inplace=True)

### Creamos una variable numerica del label para poder estudiar las posibles correlaciones del modelo 

In [9]:
encoder = OrdinalEncoder(categories=[["0-10", "11-20", "21-30", "31-40", "41-50","51-60", "61-70","71-80","81-90","91-100","More than 100 Days"]])
encoder.fit(df[["17"]])
df["18"] = encoder.transform(df[["17"]])

### Transformamos el restos de las variables categóricas.

In [10]:
df.groupby("13").mean().sort_values("18")

,3,5,9,10,11,14,16,18
13,,,,,,,,
Minor,4.628320,3.189451,2.993924,65948.145820,7.386775,3.197267,4982.036705,2.328234
Moderate,4.818332,3.210152,2.562339,65958.584727,7.151935,3.294734,4870.204126,2.749837
Extreme,4.872161,3.168705,2.251227,65943.410743,7.214514,3.447213,4753.815681,3.141987


In [11]:


encoder = OrdinalEncoder(categories=[["a", "e", "c", "b", "d","f","g"]])
encoder.fit(df[["2"]])
df["2"] = encoder.transform(df[["2"]])


encoder = OrdinalEncoder(categories=[["surgery", "radiotherapy", "gynecology", "anesthesia", "TB & Chest disease"]])
encoder.fit(df[["6"]])
df["6"] = encoder.transform(df[["6"]])

encoder = OrdinalEncoder(categories=[["U", "P", "Q", "R", "T","S"]])
encoder.fit(df[["7"]])
df["7"] = encoder.transform(df[["7"]])

encoder = OrdinalEncoder(categories=[["C", "E", "F", "A", "D","B"]])
encoder.fit(df[["8"]])
df["8"] = encoder.transform(df[["8"]])


encoder = OrdinalEncoder(categories=[["Urgent", "Emergency", "Trauma"]])
encoder.fit(df[["12"]])
df["12"] = encoder.transform(df[["12"]])

encoder = OrdinalEncoder(categories=[["Minor", "Moderate", "Extreme"]])
encoder.fit(df[["13"]])
df["13"] = encoder.transform(df[["13"]])

### Rellenamos los Nan

In [12]:
df.loc[:,'9'] =df.loc[:,'9'].fillna(df.loc[:,'9'].mean())
df.loc[:,'11'] =df.loc[:,'11'].fillna(df.loc[:,'11'].mean())

In [13]:
df.corr()

,2,3,5,6,7,8,9,10,11,12,13,14,16,18
2,1.000000,0.160653,-0.061166,0.007567,0.166113,0.667789,0.017539,-0.002297,0.055460,0.002661,0.011119,-0.021871,0.045798,0.079047
3,0.160653,1.000000,-0.043746,-0.025421,0.078908,0.140531,-0.048572,-0.004483,-0.027092,-0.029179,0.027724,0.021327,-0.033820,0.008060
5,-0.061166,-0.043746,1.000000,-0.048221,-0.412858,-0.033752,-0.118358,0.005043,-0.010329,-0.042266,-0.003820,0.091744,-0.141999,-0.122707
6,0.007567,-0.025421,-0.048221,1.000000,-0.000910,0.041826,0.098129,-0.003809,-0.019157,0.052443,-0.007247,-0.025482,0.080486,-0.033860
7,0.166113,0.078908,-0.412858,-0.000910,1.000000,0.078117,-0.019927,-0.001257,0.028416,-0.013103,0.005429,-0.074416,0.038664,0.196842
8,0.667789,0.140531,-0.033752,0.041826,0.078117,1.000000,0.042379,-0.005469,0.057429,0.023686,-0.005469,-0.012285,0.044741,0.074382
9,0.017539,-0.048572,-0.118358,0.098129,-0.019927,0.042379,1.000000,0.005769,-0.010877,0.051248,-0.286566,0.093029,0.071124,0.029385
10,-0.002297,-0.004483,0.005043,-0.003809,-0.001257,-0.005469,0.005769,1.000000,0.006709,-0.004442,-0.000015,0.007707,-0.001891,0.002740
11,0.055460,-0.027092,-0.010329,-0.019157,0.028416,0.057429,-0.010877,0.006709,1.000000,-0.031348,-0.014363,-0.008165,0.025256,-0.002476
12,0.002661,-0.029179,-0.042266,0.052443,-0.013103,0.023686,0.051248,-0.004442,-0.031348,1.000000,0.047661,0.021817,0.076126,0.082380


In [14]:
X = df[["2","5","7","10","12","13","14"]]
y = df["17"]
train_X, test_X, train_y, test_y = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [53]:

from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(train_X, train_y)
predictions = model.predict(test_X)


In [54]:
from sklearn import metrics

In [55]:
metrics.accuracy_score(test_y, predictions)

0.32915

In [56]:
test_y.shape

(20000,)

In [57]:
predictions.shape

(20000,)

-----------------------------------------------------------------

In [58]:
X_pred = pd.read_csv("hospital_test.csv",index_col=0)
X_pred.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,,,,,,,,,,,,,,,,
314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0


In [59]:
X_pred.drop(["1","4"],axis=1,inplace=True)

In [60]:
X_pred.loc[:,'9'] =X_pred.loc[:,'9'].fillna(X_pred.loc[:,'9'].mean())
X_pred.loc[:,'11'] =X_pred.loc[:,'11'].fillna(X_pred.loc[:,'11'].mean())

In [61]:

encoder = OrdinalEncoder(categories=[["a", "e", "c", "b", "d","f","g"]])
encoder.fit(X_pred[["2"]])
X_pred["2"] = encoder.transform(X_pred[["2"]])

encoder = OrdinalEncoder(categories=[["surgery", "radiotherapy", "gynecology", "anesthesia", "TB & Chest disease"]])
encoder.fit(X_pred[["6"]])
X_pred["6"] = encoder.transform(X_pred[["6"]])

encoder = OrdinalEncoder(categories=[["U", "P", "Q", "R", "T","S"]])
encoder.fit(X_pred[["7"]])
X_pred["7"] = encoder.transform(X_pred[["7"]])

encoder = OrdinalEncoder(categories=[["C", "E", "F", "A", "D","B"]])
encoder.fit(X_pred[["8"]])
X_pred["8"] = encoder.transform(X_pred[["8"]])

encoder = OrdinalEncoder(categories=[["Urgent", "Emergency", "Trauma"]])
encoder.fit(X_pred[["12"]])
X_pred["12"] = encoder.transform(X_pred[["12"]])

encoder = OrdinalEncoder(categories=[["Minor", "Moderate", "Extreme"]])
encoder.fit(X_pred[["13"]])
X_pred["13"] = encoder.transform(X_pred[["13"]])

In [62]:
X_pred = X_pred[["2","5","7","10","12","13","14"]]

In [63]:
X.shape

(100000, 7)

In [64]:
X_pred.shape

(133744, 7)

In [65]:
predictions_submit = model.predict(X_pred)
predictions_submit

array(['11-20', '21-30', '21-30', ..., '21-30', '21-30', '21-30'],
      dtype='<U18')

In [66]:
predictions_submit.shape

(133744,)

**¿Qué es lo que subirás a Kaggle?**

In [67]:
sample = pd.read_csv("sample_submission.csv")

In [68]:
sample.head()

,id,days
0,314114,11-20
1,208989,31-40
2,305872,81-90
3,266099,21-30
4,13228,31-40


In [69]:
sample.shape

(133744, 2)

In [70]:
submission = pd.DataFrame({"id": X_pred.index, "days": predictions_submit})

In [71]:
submission.head()

,id,days
0,314114,11-20
1,208989,21-30
2,305872,21-30
3,266099,21-30
4,13228,21-30


In [72]:
submission.shape

(133744, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [73]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [74]:
chequeator(submission)

You're ready to submit!
